# Adiabatic SSM model

This notebook takes multiple local SSM models which have been identified at pre-tensioned equilibria and interpolates them using scattered interpolation to obtain one adiabatic SSM model. The model performance is evaluated by its ability to predict open-loop trajectories.

In [1]:
from os import listdir
from os.path import join, isdir
import pickle
import yaml
import numpy as np
np.set_printoptions(linewidth=100)

In [2]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot
from interpolators import InterpolatorFactory

In [3]:
%matplotlib qt
import matplotlib.pyplot as plt

In [4]:
# Adiabatic SSM settings
ROMOrder = 3
# N_samples = 100
DT = 0.01
INTERPOLATE = "reduced_coords" # "xyz" # "xy" # "reduced_coords"

In [5]:
observables = "delay-embedding" # "delay-embedding" # "pos-vel" # 
N_DELAY = 4 # only relevant if observables is "delay-embedding"
TIP_NODE = 51
N_NODES = 709
INPUT_DIM = 8
DT = 0.01

rDOF = 3
oDOF = 3
SSMDim = 6

robot_dir = "../../../soft-robot-control/examples/trunk"
rest_file = join(robot_dir, 'rest_qv.pkl')

# load rest position
with open(rest_file, 'rb') as f:
    rest_data = pickle.load(f)
    rest_q = rest_data['q'][0]

In [7]:
model_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic_10ms_N=100_sparsity=0.95" # 9" # 147" # 33_handcrafted" #  # 
model_names = [name for name in sorted(listdir(model_dir)) if isdir(join(model_dir, name))]
print(model_names)

USE_DEFAULT_MODELS = True

default_model_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic_10ms_N=9"
default_model_names = [name for name in sorted(listdir(default_model_dir)) if isdir(join(default_model_dir, name))]
print(default_model_names)

N_models = len(model_names) + USE_DEFAULT_MODELS * len(default_model_names)
print("N_models:", N_models)

['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099']
['000', '001', '002', '003', '004', '005', '006', '007', '008']
N_models: 109


Compute observables (delay embedding)

In [8]:
if observables == "delay-embedding":
    N_DELAY = 4
    # observables are position of tip + n_delay delay embeddings of the tip position
    assemble_observables = lambda oData: utils.delayEmbedding(oData, up_to_delay=N_DELAY)
elif observables == "pos-vel":
    # observables is position and velocity of tip node
    def assemble_observables(oData):
        if oData.shape[0] > 6:
            tip_node_slice = np.s_[3*TIP_NODE:3*TIP_NODE+3]
        else:
            tip_node_slice = np.s_[:3]
        return np.vstack((oData[tip_node_slice, :], np.gradient(oData[tip_node_slice, :], DT, axis=1)))

Load local SSM models

In [9]:
models = []
test_results = []

if USE_DEFAULT_MODELS:
    for model_name in default_model_names:
        dir = join(default_model_dir, model_name, f"SSMmodel_{observables}_ROMOrder=3_globalV_fixed-delay")
        with open(join(dir, "SSM_model.pkl"), "rb") as f:
            model = pickle.load(f)
            models.append(model)
        with open(join(dir, "test_results.yaml"), "rb") as f:
            test_results_dict = yaml.safe_load(f)
            test_results.append([test_results_dict['like training data']['RMSE'], test_results_dict['open-loop_circle']['RMSE']])

for model_name in model_names:
    dir = join(model_dir, model_name, f"SSMmodel_{observables}_globalV")
    with open(join(dir, "SSM_model.pkl"), "rb") as f:
        model = pickle.load(f)
        models.append(model)
    with open(join(dir, "test_results.yaml"), "rb") as f:
        test_results_dict = yaml.safe_load(f)
        test_results.append([test_results_dict['like training data']['RMSE'], test_results_dict['open-loop_circle']['RMSE']])

V = [model['model']['V'] for model in models]
r_coeff = [model['model']['r_coeff'] for model in models]
w_coeff = [model['model']['w_coeff'] for model in models]
v_coeff = [model['model']['v_coeff'] for model in models]
B_r = [model['model']['B'] for model in models]
q_bar = [(model['model']['q_eq'] - rest_q)[TIP_NODE*3:TIP_NODE*3+3] for model in models]
u_bar = [model['model']['u_eq'] for model in models]
ROMOrder = models[0]['params']['ROM_order']
SSMOrder = models[0]['params']['SSM_order']
for model in models:
    assert model['params']['ROM_order'] == ROMOrder
    assert model['params']['SSM_order'] == SSMOrder
test_results = np.array(test_results)

In [10]:
if INTERPOLATE == "xyz":
    psi_eq = [q_bar[i][:3] for i in range(N_models)]
elif INTERPOLATE == "xy":
    psi_eq = [q_bar[i][:2] for i in range(N_models)]
elif INTERPOLATE == "reduced_coords":
    psi_eq = [V[i].T @ np.tile(q_bar[i], 5) for i in range(N_models)]
# psi_eq
# print(u_bar)
# print(np.sum([u == 0 for u in u_bar], axis=1))
# print(np.mean(np.sum([u == 0 for u in u_bar], axis=1)))
print(psi_eq)

[array([ 0.0116741 ,  0.09136295,  0.0383846 ,  0.00076458, -0.00433124,  0.00074858]), array([ 72.9267292 , -20.04814538, -28.61369534,   3.39920343,   1.74130791,  -1.4508973 ]), array([-32.36741758, -62.47260605, -37.32622321,  -1.50355753,   2.99363918,  -1.87813233]), array([-73.89207721,  33.43785334,   3.89146107,  -3.11097681,  -1.89676717,  -1.53847375]), array([29.37044068, 73.49785929, 12.97434427,  1.68149513, -3.06984472, -1.00990627]), array([ 41.81167278, -82.87095552, -66.49121622,   1.95456372,   4.76607361,  -3.35980064]), array([-108.11923851,  -27.69264586,  -34.11427275,   -4.68187634,    1.04114646,   -3.55186094]), array([-44.18260968, 106.98101026,  15.62895123,  -1.40198947,  -4.94621196,  -2.6718042 ]), array([101.23388332,  54.22021898, -16.27941313,   5.04466878,  -1.35641969,  -2.5637134 ]), array([ 0.0116741 ,  0.09136295,  0.0383846 ,  0.00076458, -0.00433124,  0.00074858]), array([ 39.53022762, -11.34864734, -13.69727102,   1.82163678,   0.93417083,  -0.

Interpolate local models to obtain adiabatic SSM model

In [11]:
interpolation_methods = ["origin_only", "nn", "modified_idw", "qp"]
coeff_dict = {
            'w_coeff': w_coeff,
            'V': V,
            'r_coeff': r_coeff,
            'B_r': B_r,
            'u_bar': u_bar,
        }
# if INTERPOLATE in ["xyz", "xy"]:
coeff_dict['q_bar'] = q_bar
if INTERPOLATE == "reduced_coords":
    coeff_dict['x_bar'] = [V[i].T @ np.tile(q_bar[i], 5) for i in range(len(q_bar))]

interpolators = {}
for interpolation_method in interpolation_methods:
    if interpolation_method == "qp":
        interp_slice = np.s_[:2]
    else:
        interp_slice = np.s_[:]
    interpolators[interpolation_method] = InterpolatorFactory(interpolation_method, [psi_eq[interp_slice] for psi_eq in psi_eq], coeff_dict).get_interpolator()

If interpolation is based on Delaunay triangulation, visualize the triangulation scheme

In [12]:
# if INTERPOLATION_METHOD in ["linear", "ct"] and len(xyz_rest[0]) == 2:
#     fig, ax = plt.subplots(1, 1, dpi=150)
#     ax.triplot(xyz_rest[:, 0], xyz_rest[:, 1], interpolator.tri.simplices.copy(), color="black", ls="-", lw=0.8)
#     ax.set_aspect("equal")
#     ax.set_xlabel(r"$x$ [mm]")
#     ax.set_ylabel(r"$y$ [mm]")
#     fig.suptitle("Delaunay triangulation")
#     fig.show()

For linear interpolation, visualize weights

In [13]:
# if INTERPOLATION_METHOD == "linear" and N_samples == 9:
#     nx, ny = (100, 100)
#     grid_lims = (-40, 40)
#     x = np.linspace(grid_lims[0], grid_lims[1], nx)
#     y = np.linspace(grid_lims[0], grid_lims[1], ny)
#     # xv, yv = np.meshgrid(x, y, sparse=False)

#     model_weights = np.zeros((N_models, nx, ny))
#     for i in range(nx):
#         for j in range(ny):
#             p = [x[i], y[j]]
#             simplex = interpolator.tri.find_simplex(p)
#             if simplex == -1:
#                 # extrapolation would be necessary
#                 model_weights[:, j, i] = np.nan
#             else:
#                 b = interpolator.tri.transform[simplex, :2] @ (p - interpolator.tri.transform[simplex, 2])
#                 c = np.r_[b, 1 - b.sum()]
#                 point_idx = interpolator.tri.simplices[simplex]
#                 model_weights[point_idx, j, i] = c
#     fig, axs = plt.subplots(3, 3, figsize=(10, 10), dpi=100, sharey=True, sharex=True, constrained_layout=True)
#     im = None
#     axs = axs.ravel()
#     for i, model in enumerate(model_names):
#         ax = axs[i]
#         im = ax.contourf(x, y, model_weights[i, :, :],
#                         vmin=0, vmax=1, levels=100) # cmap=mpl.colormaps[colormaps[i]])
#         # ax.set_title(model)
#         ax.set_xlabel(r"$x$ [mm]")
#         ax.set_aspect("equal")
#     for ax in axs[[0, 3, 6]]:
#         ax.set_ylabel(r"$y$ [mm]")
#     fig.colorbar(im, ax=axs.ravel().tolist(), fraction=0.03, aspect=35)
#     plt.show()

## Open-loop prediction

In [14]:
ALPHA = 0.7
traj_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/autonomous_ASSM_tests"

Import the two open-loop trajectories:
- unperturbed: should be close to ideal
- perturbed: ground truth; compare against advection

In [15]:
# unperturbed
(t_unperturbed, z_unperturbed), u_unperturbed = utils.import_pos_data(data_dir=join(traj_dir, "OL_sim_ideal", f"alpha={ALPHA:.1f}"),
                                  rest_file=rest_file,
                                  output_node=TIP_NODE, return_inputs=True, t_in=3)
y_unperturbed = assemble_observables(z_unperturbed)
print("y.shape:", y_unperturbed.shape)
print("z.shape:", z_unperturbed.shape)
print("u.shape:", u_unperturbed.shape)
# perturbed
(t_perturbed, z_perturbed), u_perturbed = utils.import_pos_data(data_dir=join(traj_dir, "OL_sim_perturbed", f"alpha={ALPHA:.1f}"),
                                  rest_file=rest_file,
                                  output_node=TIP_NODE, return_inputs=True, t_in=3)
y_perturbed = assemble_observables(z_perturbed)
print("y.shape:", y_perturbed.shape)
print("z.shape:", z_perturbed.shape)
print("u.shape:", u_perturbed.shape)

y.shape: (15, 899)
z.shape: (3, 899)
u.shape: (8, 899)
y.shape: (15, 899)
z.shape: (3, 899)
u.shape: (8, 899)


Ideal target trajectory: circle starting at $\varphi=\frac{\pi}{2}$

In [16]:
z_eq = np.array([3.54595786, 0.30603326, -195.61577293])

M = 1
T = len(t_unperturbed) * DT # 9 # 10
N = len(t_unperturbed) # 1000
radius = 30.
t_target = np.linspace(0, M * T, M * N + 1)
th = np.linspace(0, M * 2 * np.pi, M * N + 1) + np.pi/2
zf_target = np.tile(z_eq, (M * N + 1, 1))
# zf_target = np.zeros((M*N+1, 6))
zf_target[:, 0] += radius * np.cos(th)
zf_target[:, 1] += radius * np.sin(th)

z_target = zf_target - z_eq
y_target = assemble_observables(z_target.T)

Plot: (a) ideal target, (b) unperturbed open-loop sim, (c) perturbed open-loop sim

In [17]:
plt.close("all")
# Trajectories xyz vs. t
axs = plot.traj_xyz_txyz(t_unperturbed,
                         y_unperturbed[0, :], y_unperturbed[1, :], y_unperturbed[2, :],
                         show=False)
axs = plot.traj_xyz_txyz(t_perturbed,
                         y_perturbed[0, :], y_perturbed[1, :], y_perturbed[2, :],
                         color="tab:orange", axs=axs, show=False)
axs = plot.traj_xyz_txyz(t_target,
                         z_target[:, 0], z_target[:, 1], z_target[:, 2],
                         color="k", ls="--", axs=axs, show=False)
axs[-1].legend(["Unperturbed", "Perturbed", "Target"])
plt.suptitle(r"Trajectories")
plt.tight_layout()
plt.show()

# Trajectories 2D (predicted vs. actual)
ax = plot.traj_2D_xy(y_unperturbed[0, :], y_unperturbed[1, :],
                     show=False)
ax = plot.traj_2D_xy(y_perturbed[0, :], y_perturbed[1, :],
                     color="tab:orange", ax=ax, show=False)
ax = plot.traj_2D_xy(z_target[:, 0], z_target[:, 1],
                     color="k", ls="--", ax=ax, show=False)
ax.legend(["Unperturbed", "Perturbed", "Target"])
plt.suptitle(r"Trajectories")
plt.show()

### Predict by advecting SSM models

Then advect the perturbed case using the different SSM models

In [31]:
for interpolation_method in interpolation_methods:
    print(f"========== {interpolation_method} ==========")
    if interpolation_method == "qp":
        interp_slice = np.s_[:2]
    else:
        interp_slice = None
    t, x, y_pred, xdot, y_bar, u_bar, weights = utils.advect_adiabaticRD_with_inputs(t_unperturbed, y_perturbed[:, 0], u_unperturbed, y_unperturbed,
                                                                                     interpolator=interpolators[interpolation_method], know_target=True, ROMOrder=ROMOrder,
                                                                                     interpolate=INTERPOLATE, interp_slice=interp_slice)
    assert len(t) == len(t_perturbed)
    # compute RMSE
    z_pred = y_pred[-3:, :]
    rmse = np.sqrt(np.mean(np.linalg.norm(z_pred - z_perturbed, axis=0)**2))
    print(rmse)
    print(f"RMSE = {rmse:.4f}")
    # save the predicted trajectory
    with open(join(traj_dir, "predicted_trajectories", f"alpha={ALPHA:.1f}", f"z_pred_{interpolation_method}.pkl"), "wb") as f:
        pickle.dump(z_pred, f)

========== origin_only ==========
9.804190515121029
RMSE = 9.8042
========== nn ==========
1.9592320645118404
RMSE = 1.9592
========== modified_idw ==========
1.2655493132740823
RMSE = 1.2655
========== qp ==========
0.951058036996236
RMSE = 0.9511


### Predict by advecting baseline models (TPWL, Koopman)

#### Koopman

In [32]:
from scipy.io import loadmat
from sofacontrol.baselines.koopman import koopman_utils

In [33]:
koopman_data = loadmat("/home/jonas/Projects/stanford/soft-robot-control/examples/trunk/koopman_model.mat")['py_data'][0, 0]
raw_model = koopman_data['model']
raw_params = koopman_data['params']
model = koopman_utils.KoopmanModel(raw_model, raw_params)
scaling = koopman_utils.KoopmanScaling(scale=model.scale)

Advect Koopman model and evaluate model accuracy

In [34]:
z_perturbed_koopman = (z_perturbed.T + rest_q[3*TIP_NODE:3*(TIP_NODE+1)]).T
z_perturbed_koopman = scaling.scale_down(y=z_perturbed_koopman.T).T
u_unperturbed_koopman = scaling.scale_down(u=utils.delayEmbedding(u_unperturbed, up_to_delay=1, embed_coords=list(range(8)))[:8, :].T).T
y_perturbed_koopman = np.vstack([z_perturbed_koopman, utils.delayEmbedding(z_perturbed_koopman, up_to_delay=1)[:3, :], u_unperturbed_koopman])

y0 = y_perturbed_koopman[:, 0]
N = len(t_perturbed) - 1
x = np.full((model.A_d.shape[0], N+1), np.nan)
xdot = np.full((model.A_d.shape[0], N), np.nan)
y_pred = np.full((len(y0), N+1), np.nan)
y_pred[:, 0] = y0
# advect for horizon N
for i in range(N):
    # lift the observables
    if i == 0:
        x[:, i] = model.lift_data(*y_pred[:, i])
    # compute the Koopman prediction
    # xdot[:, i]
    x[:, i+1] = model.A_d @ x[:, i] + model.B_d @ u_unperturbed_koopman[:, i]
    # forward Euler: x[i+1] = x[i] + dt * xdot[i]
    # x[:, i+1] = x[:, i] + DT * xdot[:, i]
    y_pred[:, i+1] = np.concatenate([model.C @ x[:, i+1], y_pred[:3, i], u_unperturbed_koopman[:, i]])
# compute RMSE
z_pred = (scaling.scale_up(y=y_pred[:3, :].T) - rest_q[3*TIP_NODE:3*(TIP_NODE+1)]).T
rmse = np.sqrt(np.mean(np.linalg.norm(z_pred - z_perturbed, axis=0)**2))
print(f"========== Koopman ==========")
print(f"RMSE = {rmse:.4f}")
# save the predicted trajectory
with open(join(traj_dir, "predicted_trajectories", f"z_pred_koopman.pkl"), "wb") as f:
    pickle.dump(z_pred, f)

========== Koopman ==========
RMSE = 8.2679


#### TPWL

In [35]:
from sofacontrol.tpwl import tpwl, tpwl_config
from sofacontrol.measurement_models import linearModel, MeasurementModel

# Specify a measurement and output model
cov_q = 0.0 * np.eye(3 * len([TIP_NODE]))
cov_v = 0.0 * np.eye(3 * len([TIP_NODE]))
measurement_model = linearModel(nodes=[TIP_NODE], num_nodes=N_NODES)
output_model = MeasurementModel(nodes=[TIP_NODE], num_nodes=N_NODES, pos=True, vel=True, S_q=cov_q, S_v=cov_v)
# Load and configure the TPWL model from data saved
tpwl_model_file = "/home/jonas/Projects/stanford/soft-robot-control/examples/trunk/tpwl_model_snapshots.pkl"
config = tpwl_config.tpwl_dynamics_config()
model = tpwl.TPWLATV(data=tpwl_model_file, params=config.constants_sim, Hf=output_model.C,
                     Cf=measurement_model.C)
model.pre_discretize(dt=DT)

TPWL num linearization points 2305
Performing pre-discretization using implicit Euler of TPWL model with dt = 0.010


In [36]:
print(measurement_model.C.shape)
print(model.C.shape)

(6, 4254)
(6, 28)


Import trajectories (pos and vel) of all FEM nodes

In [37]:
# unperturbed
(t_unperturbed_tpwl, qv_unperturbed_tpwl), u_unperturbed_tpwl = utils.import_pos_data(data_dir=join(traj_dir, "OL_sim_ideal"),
                                  rest_file=rest_file,
                                  output_node="all", return_inputs=True, return_velocity=True, t_in=2)
print("z.shape:", qv_unperturbed_tpwl.shape)
print("u.shape:", u_unperturbed_tpwl.shape)
# perturbed
(t_perturbed_tpwl, qv_perturbed_tpwl), u_perturbed_tpwl = utils.import_pos_data(data_dir=join(traj_dir, "OL_sim_perturbed"),
                                  rest_file=rest_file,
                                  output_node="all", return_inputs=True, return_velocity=True, t_in=2)

q_perturbed_tpwl = qv_perturbed_tpwl[:3*N_NODES, :]
v_perturbed_tpwl = qv_perturbed_tpwl[3*N_NODES:, :]
q_unperturbed_tpwl = qv_unperturbed_tpwl[:3*N_NODES, :]
v_unperturbed_tpwl = qv_unperturbed_tpwl[3*N_NODES:, :]

z.shape: (4254, 902)
u.shape: (8, 902)


Advect TWPL model and evaluate model accuracy

In [38]:
xf = np.vstack([v_perturbed_tpwl, q_perturbed_tpwl]) + model.rom.x_ref[:, None]
# print(xf.shape)
x0 = model.rom.compute_RO_state(xf=xf[:, 0])
x = np.full((model.state_dim, N+1), np.nan)
xdot = np.full((model.state_dim, N), np.nan)
x[:, 0] = x0
z0 = model.x_to_zy(x0, z=True)
# print(z0)
# print(z_tot[:, start_idx])
# advect for horizon N
for i in range(N):
    # compute the TPWL prediction
    A, B, d = model.get_jacobians(x=x[:, i], u=u_unperturbed_tpwl[:, i], dt=DT)
    # xdot[i] = R(x[i]) + B[i] @ (u[i] - u_bar[i])
    # xdot[:, i] = A @ x[:, i] + B @ u[:, i] + d
    # print(np.linalg.eig(A))
    # forward Euler: x[i+1] = x[i] + dt * xdot[i]
    x[:, i+1] = A @ x[:, i] + B @ u_unperturbed_tpwl[:, i] + d
    # x[:, i+1] = x[:, i] + DT * xdot[:, i]
# compute RMSE
z_pred = model.x_to_zy(x.T, z=True).T[3:, :]
rmse = np.sqrt(np.mean(np.linalg.norm(z_pred - z_perturbed, axis=0)**2))

print(f"========== TPWL ==========")
print(f"RMSE = {rmse:.4f}")
# save the predicted trajectory
with open(join(traj_dir, "predicted_trajectories", f"z_pred_tpwl.pkl"), "wb") as f:
    pickle.dump(z_pred, f)

========== TPWL ==========
RMSE = 9.0324


## Plot results

In [36]:
display_names = {
    "origin_only": "Origin SSM",
    # "linear": "Adiabatic SSM (linear)",
    "ct": "Clough-Tocher",
    "nn": "NN",
    "idw": "IDW", # (inverse distance weighting)
    "modified_idw": "MIDW",
    "qp": "Adiabatic SSM", # "QPR", # (quadratic polynomial regression)
    "tpwl": "TPWL",
    "koopman": "Koopman",
}
colors = {
    "origin_only": "tab:blue",
    # "linear": "tab:orange",
    "ct": "tab:pink",
    "idw": "tab:pink",
    "modified_idw": "tab:red",
    "nn": "tab:brown",
    "qp": "tab:purple",
    "tpwl": "tab:green",
    "koopman": "tab:orange",
}

lw = {
    "origin_only": .8,
    # "linear": 1,
    "ct": 2,
    "idw": 2,
    "modified_idw": 2,
    "nn": 2,
    "qp": 4,
    "tpwl": .8,
    "koopman": .8,
}

markers = {
    "origin_only": ("o", None),
    # "linear": None,
    "ct": (None, None),
    "idw": (None, None),
    "modified_idw": ("v", None),
    "nn": ("^", None),
    "qp": ("D", 7),
    "tpwl": ("v", None),
    "koopman": ("^", None),
}

### Plot trajectories predicted by different SSM models and baselines

Plot for current alpha = ALPHA

In [37]:
plt.close('all')

axs = plot.traj_xyz_txyz(t_perturbed,
                         y_perturbed[0, :], y_perturbed[1, :], y_perturbed[2, :],
                         show=False, color="k", ls="--", label="Perturbed trajectory") # (ground truth)
ax = plot.traj_2D_xy(y_perturbed[0, :], y_perturbed[1, :],
                     show=False, color="k", ls="--", label="Perturbed trajectory") # (ground truth)
axs = plot.traj_xyz_txyz(t_perturbed,
                         y_unperturbed[0, :], y_unperturbed[1, :], y_unperturbed[2, :],
                         axs=axs, show=False, color="#8a8a8a", ls="--", label="Unperturbed trajectory") # (ground truth)
ax = plot.traj_2D_xy(y_unperturbed[0, :], y_unperturbed[1, :],
                     ax=ax, show=False, color="#8a8a8a", ls="--", label="Unperturbed trajectory") # (ground truth)
ax.plot(y_perturbed[0, 0], y_perturbed[1, 0], 'o', markersize=8, color="k")

for model in ["origin_only", "qp", "tpwl", "koopman"]:
        # Trajectory xyz vs. t (predicted vs. actual)
        with open(join(traj_dir, "predicted_trajectories", f"alpha={ALPHA:.1f}", f"z_pred_{model}.pkl"), "rb") as f:
            y_pred = pickle.load(f)
        axs = plot.traj_xyz_txyz(t_unperturbed,
                                y_pred[0, :], y_pred[1, :], y_pred[2, :],
                                color=colors[model], axs=axs, show=False,
                                lw=lw[model], marker=markers[model][0], markersize=markers[model][1], label=display_names[model])
        # Trajectory 2D (predicted vs. actual)
        ax = plot.traj_2D_xy(y_pred[0, :], y_pred[1, :],
                             color=colors[model], ax=ax, show=False,
                             lw=lw[model], marker=markers[model][0], markersize=markers[model][1], label=display_names[model])
# axs[-1].legend()
# axs[-1].get_figure().suptitle(rf"Trajectories (true vs. predicted) -- ROMOrder={rom_order}")
axs[-1].get_figure().savefig(join(traj_dir, f"open-loop-prediction-xyz-vs-t.svg"), dpi=300, bbox_inches='tight')
axs[-1].get_figure().savefig(join(traj_dir, f"open-loop-prediction-xyz-vs-t.eps"), dpi=300, bbox_inches='tight')
axs[-1].get_figure().show()
ax.legend(loc="lower left")
ax.grid(True, color="#cccccc")
ax.set_xlim(-45, 45)
ax.set_ylim(-45, 45)
# ax.get_figure().suptitle(rf"Trajectories (true vs. predicted) -- ROMOrder={rom_order}")
ax.get_figure().savefig(join(traj_dir, f"open-loop-prediction_x-vs-y.svg"), dpi=300, bbox_inches='tight')
ax.get_figure().savefig(join(traj_dir, f"open-loop-prediction_x-vs-y.eps"), dpi=300, bbox_inches='tight')
ax.get_figure().show()

### Plot trajectories predicted by Adiabatic SSM (QP) for different alphas

In [20]:
model = "qp"

In [21]:
plt.close('all')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

plot.traj_2D_xy(y_unperturbed[0, :], y_unperturbed[1, :], ax=ax1,
                show=False, color="#8a8a8a", ls="--") # (ground truth)
plot.traj_2D_xy(y_unperturbed[0, :], y_unperturbed[1, :], ax=ax2,
                show=False, color="#8a8a8a", ls="--") # (ground truth)

# discretize magma cmap into 5 colors
cmap = plt.cm.magma
clrs = [cmap(i) for i in np.linspace(0.2, .8, 5)]

for i, alpha in enumerate([0.1, 0.2, 0.7, 1.5, 3.0]):
        (t_perturbed, z_perturbed), u_perturbed = utils.import_pos_data(data_dir=join(traj_dir, "OL_sim_perturbed", f"alpha={alpha:.1f}"),
                                  rest_file=rest_file,
                                  output_node=TIP_NODE, return_inputs=True, t_in=3)
        plot.traj_2D_xy(z_perturbed[0, :], z_perturbed[1, :], ax=ax1,
                        show=False, color=clrs[i], alpha=1, ls="--") # label="Perturbed trajectory" # (ground truth)
        # ax.plot(z_perturbed[0, 0], z_perturbed[1, 0], 'o', markersize=8, color="k")
        plot.traj_2D_xy(z_perturbed[0, :], z_perturbed[1, :], ax=ax2,
                        show=False, color=clrs[i], alpha=1, ls="--") # label="Perturbed trajectory" # (ground truth)
        with open(join(traj_dir, "predicted_trajectories", f"alpha={alpha:.1f}", f"z_pred_{model}.pkl"), "rb") as f:
            z_pred = pickle.load(f)
        # Trajectory 2D (predicted vs. actual)
        plot.traj_2D_xy(z_pred[0, :], z_pred[1, :],
                        color=clrs[i], ax=ax1, show=False,
                        lw=2, marker="", label=rf"$\alpha={alpha:.1f}$")
        plot.traj_2D_xy(z_pred[0, :], z_pred[1, :],
                        color=clrs[i], ax=ax2, show=False,
                        lw=2, marker="", label=rf"$\alpha={alpha:.1f}$")
# ax1.legend(loc="lower left")
ax1.grid(True, color="#cccccc")
ax1.set_xlim(-35, 35)
ax1.set_ylim(-35, 41)
ax2.legend(loc="upper left")
ax2.grid(True, color="#cccccc")
ax2.set_xlim(-19, 1)
ax2.set_ylim(20, 40)

fig.savefig(join(traj_dir, f"open-loop-prediction_{model}_different-alphas_x-vs-y.png"), dpi=300, bbox_inches='tight')
fig.show()

In [22]:
plt.close("all")
# plot RSMEs for each alpha
fig, ax = plt.subplots(1, 1, figsize=(12, 5))

alphas = [0.1, 0.2, 0.4, 0.7, 1.0, 1.5, 2.0, 3.0]
for model in ["origin_only", "qp", "modified_idw", "nn"]:
    rmses = []
    for alpha in alphas:
        with open(join(traj_dir, "predicted_trajectories", f"alpha={alpha:.1f}", f"z_pred_{model}.pkl"), "rb") as f:
            z_pred = pickle.load(f)
        (t_perturbed, z_perturbed), u_perturbed = utils.import_pos_data(data_dir=join(traj_dir, "OL_sim_perturbed", f"alpha={alpha:.1f}"),
                                    rest_file=rest_file,
                                    output_node=TIP_NODE, return_inputs=True, t_in=3)
        rmse = np.sqrt(np.mean(np.linalg.norm(z_pred - z_perturbed, axis=0)**2))
        rmses.append(rmse)
    ax.plot(alphas, rmses, lw=2, ls='-', marker=markers[model], color=colors[model], label=display_names[model])

# make it a loglog plot
ax.set_xscale("log")
ax.set_yscale("log")
from matplotlib.ticker import ScalarFormatter
ax.xaxis.set_major_formatter(ScalarFormatter())
ax.yaxis.set_major_formatter(ScalarFormatter())
ax.set_xticks(alphas)
ax.set_yticks([.6, .8, 1.0, 2.0, 3.0, 4.0])
# ax.set_xlim([0.1, 3])
# ax.set_ylim([0.6, 4.5])
ax.grid(True, color="#cccccc")
ax.legend(loc="best")
ax.set_ylabel("RMSE [mm]")
ax.set_xlabel(r"$\alpha\ =\ \frac{2 \pi}{T}$")

fig.savefig(join(traj_dir, f"open-loop-prediction_different-alphas_rmses.png"), dpi=300, bbox_inches='tight')
fig.show()